# Lab | API wrappers - SPOTIFY 

In [ ]:
import pandas as pd

from bs4 import BeautifulSoup

# access information on any webpage...

from bs4 import BeautifulSoup
import requests

## Authentification

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials # to create a connection to spotify

#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='<your client id here>',
#                                                           client_secret='<your client secret here>'))

## Storing secrets

In [ ]:
secrets_file = open("secrets.txt","r")

In [ ]:
string = secrets_file.read()

In [ ]:
string

In [ ]:
string.split('\n')

In [ ]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        print(line.split(': '))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [ ]:
secrets_dict

## Authentication with secrets

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

## Web scraping - Spotify

In [ ]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id) # read first page
    tracks = results['items'] # assign items to a list called tracks
    while results['next']!=None: 
        results = sp.next(results) # read the next page
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # unit is second
    return tracks

# concat all items list on each page

In [ ]:
# this will take at least around num_songs_in_playlist * (avg_sleep_time + processing_time) = 53 * (2+0.1) = 110 seconds

all_tracks_1 = get_playlist_tracks("4pbDDX7np7Q1H0ghL7U9o7")
len(all_tracks_1)

In [ ]:
all_tracks_2 = get_playlist_tracks("532F1h299qKD894BlPfJJF")
len(all_tracks_2)

In [ ]:
def playlist_df_creator(owner, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name", "track_id","danceability","energy","key","loudness","mode", "speechiness","acousticness","instrumentalness","liveness","valence","tempo","type","id","uri","track_href","analysis_url", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    all_tracks = get_playlist_tracks(playlist_id)
    # Explore every track in the playlist, extract features and append the features to the playlist_df
#     playlist = sp.user_playlist_tracks(owner, playlist_id)
#     tracks = playlist['items']
    
    for track in all_tracks:
        
        # Create empty dictionary
        playlist_features = {}
        
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
playlist_df_1 = playlist_df_creator("spotify", "4pbDDX7np7Q1H0ghL7U9o7")

In [ ]:
playlist_df_2 = playlist_df_creator("spotify", "532F1h299qKD894BlPfJJF")

In [ ]:
multiple_playlist_df = pd.concat([playlist_df_1, playlist_df_2], ignore_index = True)

In [ ]:
multiple_playlist_df

In [ ]:
multiple_playlist_df.shape

In [ ]:
multiple_playlist_df.to_csv('multiple_playlist_df.csv')

In [ ]:
import random

secure_random = random.SystemRandom()
print(secure_random.choice(multiple_playlist_df['track_name']))

In [ ]:
favorite_song = input('Please enter your song name: ')


In [ ]:
print(favorite_song)

In [ ]:
# if favorite_song in songs['song']:
if len(playlist_df_all[(multiple_playlist_df['track_name'].isin([favorite_song]))])>0:
    print("Our song recommendation is: ", secure_random.choice(multiple_playlist_df['track_name'])) 
else:
    print("We are sorry that we do not have a song recommendation for you now.")

In [ ]:
import random


secure_random = random.SystemRandom()
print(secure_random.choice(multiple_playlist_df['track_name']))